In [ ]:
import pandas as pd

### A partir d'un dataFrame cette methode va récuperer toute les cartes joués et en faire des phrases qui seront en suite passé à l'algorithme word2vec

### A games 
### On prend la colonne games

### A partir de card_history
### on recupere l'historique des cartes sorties pour chaque partie On filtre si le champs card_history existe ou si il est vide

### A partir de sentances
### Pour chaque partie on rassemble toutes les cartes sorties par "me" et "opponent", puis on recupère le nom des cartes pour crée des phrases.


In [ ]:
def createsentances(data):
    games = data["games"]
    card_history = []
    for i in range(len(games)):
        g = games[i]
        if not(g.has_key("card_history")) or g["card_history"] != []:
            card_history.append(pd.DataFrame(g["card_history"]))
    sentances = []
    for i in range(len(card_history)):
        c = card_history[i]
        me = c[c.player=="me"]
        op = c[c.player=="opponent"]
        me = me["card"]
        op = op['card']
        sentances.append([i["name"] for i in me])
        sentances.append([i["name"] for i in op])
    return sentances

creation de phrase a partir de plusieurs fichier json
listfile est une liste de fichier avec leur chemin

In [ ]:
def createsentancesfiles(listfile):
    sentances = []
    for e in listfile:
        data=pd.read_json(listfile)
        sentances += createsentances(data)
    return sentances

In [ ]:
directoryHearthstoneGames = "HeathstoneGames/"
from os import listdir
from os.path import isfile, join
filesHearthstoneGames = [directoryHearthstoneGames+f for f in listdir(directoryHearthstoneGames) if isfile(join(directoryHearthstoneGames, f))]


sentances = createsentancesfiles(filesHearthstoneGames)

In [ ]:
data = pd.read_json("2018-01.zip");

In [ ]:
sentances = createsentances(data)

In [ ]:
sentances = []

In [ ]:
for e in filesHearthstoneGames:
    data = pd.read_json(e);
    sentances +=createsentances(data)
    


# Je prend l'url avec collectible car je ne veux que les cartes disponibles pour faire un deck

In [36]:
import requests

urlApi = "https://api.hearthstonejson.com/v1/latest/enUS/cards.collectible.json"

ApiHearthStone = requests.get(urlApi).json()

In [ ]:
listNameCard = [e["name"] for e in ApiHearthStone]

on filtre les cartes des parties à seulement celle disponible pour la construction de deck

In [ ]:
sentances = [filter(lambda x: x in listNameCard, e) for e in sentances]

# On va utilisier gensim et plus particulièrement l'algo Word2vec

In [1]:
from gensim.models import Word2Vec

max_count exclue les mots dont leur nombre d'apparition est inférieure à celui indiqué
size nombre de noeuds
workers paralléliser sur un nombres de coeurs
ier nombre d'epochs

In [ ]:
model = Word2Vec(sentancesFiltered,min_count=10,size=200,workers=4,iter=100)

sauvegarde du model

In [ ]:
model.save("word2vecModel")

charger le modele entrainer et de continuer l'entrainement

In [3]:
model = Word2Vec.load("word2vecModel")

In [34]:
model.wv.most_similar(positive=["Open the Waygate","Fireball","The Lich King","Primordial Glyph","Ice Block"],topn=5)

[(u'Arcane Intellect', 0.9318734407424927),
 (u'Shatter', 0.9219666719436646),
 (u'Arcanologist', 0.9053369760513306),
 (u'Freezing Potion', 0.8976438045501709),
 (u'Mana Bind', 0.8919025659561157)]

In [56]:
model.wv.most_similar(positive=["Jaina Proudmoore"],topn=5)

KeyError: "word 'Jaina Proudmoore' not in vocabulary"

In [66]:
heroes = [e["name"] for e in ApiHeartStone if e["type"]=="HERO"]
filter(lambda x: x in heroes, model.wv.vocab.keys())

[u'Scourgelord Garrosh',
 u'Frost Lich Jaina',
 u'Deathstalker Rexxar',
 u'Shadowreaper Anduin',
 u'Malfurion the Pestilent',
 u'Uther of the Ebon Blade',
 u"Bloodreaver Gul'dan",
 u'Valeera the Hollow',
 u'Thrall, Deathseer']

In [64]:
model.wv.vocab.keys()

list

proposition de cartes si deck 30 ou plus aucune proposition

verifier deck depart un ou moins hero et proposition eliminer autre proposer pas besoin car present dans le dictionnaire
negative si cardClass différent 
meme classes de carte que le heros sauf neutral

In [ ]:
def proposeCards(uncompletedDeck,model,apiHS):
    cardClass = ""
    if len(uncompletedDeck) >= 30:
        return
    for e in uncompletedDeck:
        if e in "HERO":
            heroIsPresent = True
            heroClass = ["clardClass"]
        else:
    
    
    model.wv.most_similar(uncompletedDeck,topn=5)